In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from numba import jit

In [45]:
df = pd.read_csv('fake reviews dataset.csv')

In [44]:
@jit
def clean_text(text):
    nopunc = [w for w in text if w not in string.punctuation]
    nopunc = ''.join(nopunc)
    return  ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [46]:
df['text_']=df['text_'].apply(clean_text)

In [47]:
@jit
def preprocess(text):
    return ' '.join([word for word in word_tokenize(text) if word not in stopwords.words('english') and not word.isdigit() and word not in string.punctuation])

In [48]:
df['text_']=df['text_'].apply(preprocess)

In [49]:
stemmer = PorterStemmer()
@jit
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [51]:
df['text_']=df['text_'].apply(stem_words)

In [36]:
lemmatizer = WordNetLemmatizer()
@jit
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [52]:
df['text_']=df['text_'].apply(lemmatize_words)

In [53]:
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort love itveri pretti
1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin ive mine coupl year
2,Home_and_Kitchen_5,5.0,CG,pillow save back love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price
4,Home_and_Kitchen_5,5.0,CG,nice set good qualiti set two month


In [7]:
def text_process(review):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [54]:
df.dropna(inplace=True)

In [55]:
df['length'] = df['text_'].apply(len)

In [56]:
df.groupby('label').describe()

rating                                                length  \
         count      mean       std  min  25%  50%  75%  max    count   
label                                                                  
CG     20216.0  4.259893  1.141065  1.0  4.0  5.0  5.0  5.0  20216.0   
OR     20216.0  4.253265  1.147652  1.0  4.0  5.0  5.0  5.0  20216.0   

                                                                
             mean         std  min   25%    50%    75%     max  
label                                                           
CG     154.059953  155.189176  0.0  51.0   88.0  192.0  1191.0  
OR     219.051247  233.256478  8.0  68.0  123.0  272.0  1868.0

In [57]:
df.head()

,category,rating,label,text_,length
0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort love itveri pretti,48
1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin ive mine coupl year,44
2,Home_and_Kitchen_5,5.0,CG,pillow save back love look feel pillow,38
3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price,35
4,Home_and_Kitchen_5,5.0,CG,nice set good qualiti set two month,35


In [58]:
review_train, review_test, label_train, label_test = train_test_split(df['text_'],df['label'],test_size=0.35)

In [59]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])

In [60]:
pipeline.fit(review_train, label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x0000027A03CC5DC0>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [61]:
pred = pipeline.predict(review_test)

In [62]:
print(pred)


['OR' 'OR' 'CG' ... 'CG' 'CG' 'OR']


In [63]:
from sklearn.metrics import accuracy_score
print('Accuracy Score:',accuracy_score(label_test,pred)*100)

Accuracy Score: 88.0864895421142


In [70]:
txt = input()
txt = clean_text(txt)
txt = preprocess(txt)
txt = stem_words(txt)
txt = lemmatize_words(txt)
txt = [txt]
if pipeline.predict(txt)=='OR':
    print('Original')
else:
    print('Fake')

Fake


In [ ]:
df.to_csv('Preprocessed Fake Reviews Dataset.csv')